# Загрузка данных


In [ ]:
pip install face_recognition

In [ ]:
import os
import pickle
import sys
import face_recognition
from cv2 import cv2
from google.colab.patches import cv2_imshow

from IPython.display import HTML
from base64 import b64encode

from PIL import Image, ImageDraw

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
directory = '/content/drive/MyDrive/Colab Notebooks/Neural_university/Соревнование/Carrey/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Тренировка на фото

Предварительно из видео вырезаны примеры изображений Керри и помещены в папку "img_carrey".
Задача - создать файл с обобщенными данными на изображение  Керри carrey_encodings.pickle


In [ ]:
# Скрипт получает изображение, делает скриншот и отправляет на сервер. Сравниват с базой и если сопадает то делает действие
# Имя человека, которого определяем
def train_model_by_img(name):

    if not os.path.exists(directory  + "img_carrey"):
        print("[ERROR] there is no directory 'img_carrey'")
        sys.exit()

    known_encodings = []
    num_img_carrey = 0
    # Здесь несколько снимков одного человека img_carrey
    images = os.listdir(directory  + "img_carrey")
    print('Кадров для обучения:', len(images))   # Выведем имена файлов
    
    # Пробежимся по каждому изображени, загружакем в face_recognitions, снимаем кодировку и сравниваем
    # enumerate позволит кроме изображения получить его индекс - для выврода в терминал чтобы видетиь програсс
    for (i, image) in enumerate(images):     
        # print(f"[+] распознаем {i} изображение, файл - {image} из {len(images)}")
        face_img = face_recognition.load_image_file(directory  + f"img_carrey/{image}")    #Загрузка изображдений        
        try:
          face_enc = face_recognition.face_encodings(face_img)[0]      #Кодировка каждого лица

          # Каждая полученная кодировка должна сопосталяться с полученной ранее
          # Если True - то добавляем в список
          # перовначально он пустой и добавляем туда первую кодировку
          if len(known_encodings) == 0:
              known_encodings.append(face_enc)
          
          # Если там уже естm первая кодировка, то проходим по этой кодироdке
          else:
            # face_enc - новое изображение сравниваем соо всеми в списке. Может быт долго если длинный список
              for item in range(0, len(known_encodings)):        
                  result = face_recognition.compare_faces([face_enc], known_encodings[item])
                  #print(result)      # Выводим узнал или нет Керри

                  if result[0]:
                      known_encodings.append(face_enc)     # Если True - добавляем в список и выходим из списка
                      #print("Тот же человек")
                      num_img_carrey += 1
                      break
                  else:
                      #print("Другой человек!")
                      break
        except:
         print('не узнал')
    #print(known_encodings)
    #print(f"Length {len(known_encodings)}")
    print('Узнал фоток', num_img_carrey)
    data = {
        "name": name,
        "encodings": known_encodings
    }

    with open(directory + f"{name}_encodings.pickle", "wb") as file:    # wb - запись в двоичном формате
        file.write(pickle.dumps(data))

    return f"[INFO] File {name}_encodings.pickle successfully created"



# Вывод исходного видео

In [ ]:
# Вывод видео в колабе
video_path = directory + "Shou_Trumana_Kinosimka1.mp4"
mp4 = open(video_path,'rb').read()
decoded_vid = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f'<video width=400 controls><source src={decoded_vid} type="video/mp4"></video>')

Output hidden; open in https://colab.research.google.com to view.

# Распознавание человека

In [ ]:
# Определим Керри на видео и сохраним эти кадры

def detect_person_in_video(name_video):
    data = pickle.loads(open(directory + 'carrey_encodings.pickle', "rb").read())    # Читаем в двоичном формате
    video = cv2.VideoCapture(directory + name_video)          # Можноб сюда данные из камеры
    num = 0

    while True:
        ret, image = video.read()              # Читаем и декодируем кадр
        
        # cnn - для работы на gpu
        locations = face_recognition.face_locations(image, model="cnn")           # Получим массив координат лиц на кадре
        encodings = face_recognition.face_encodings(image, locations)             # получим кодировки лиц из кадров
 
        # Сравнение лиц из датасета с лицами из каждого кадра. Если совпадает - выделяем лицо на видео
        # Циклом проходим по кодировкам и локациям
        for face_encoding, face_location in zip(encodings, locations):         
            result = face_recognition.compare_faces(data["encodings"], face_encoding)    # Для сравнения кодировки лиц. Первое - из датасета, вторая из кадра
            match = None                          # Совпадение

            if True in result:                  # Если совпадает
                match = data["name"]
                #print(f"Он найден! {match}")
                cv2.imwrite(directory + f"frame_carrey/carrey{num}.jpg", image)
            #else:
                #print("Увы!")
        num += 1
        

# detect_person_in_video("Shou_Trumana_Kinosimka1.mp4")


# Создание видео и кадров

In [ ]:
# Создадим видео и кадров с Керри
def save_video(name_out_video):
  num_images = os.listdir(directory  + "frame_carrey")   # загрузим все файлы скриншотов

  #fourcc = cv2.VideoWriter_fourcc('M','J','P','G')
  fourcc = cv2.VideoWriter_fourcc(*'MJPG')

  video = cv2.VideoCapture(directory + 'Shou_Trumana_Kinosimka.RU (online-video-cutter.com).mp4') 
  fps = video.get(cv2.CAP_PROP_FPS)
  width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
  height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))

  out = cv2.VideoWriter(directory + name_out_video, fourcc, fps, (width, height)) #создаем видео
  for i in num_images:
    out.write(cv2.imread(directory + f'frame_carrey/{i}'))  #добавляем картинки из директории

  out.release() #генерируем
  cv2.destroyAllWindows() #завершаем

  return

save_video('out_video_carry6.avi')

# Главная программа

In [ ]:
def main():
    # Файл видео для распознавания
    name_video = "Shou_Trumana_Kinosimka.RU (online-video-cutter.com).mp4"
    print('Тренируем скрипт')
    train_model_by_img('carrey')
    try:
      print('Распознаем Керри')
      detect_person_in_video(name_video)
    except:
      print('все')
    
    print('Записываем видео')
    save_video('out_video_carry6.avi')




if __name__ == '__main__':
    main()


Тренируем скрипт
Кадров для обучения: 28
не узнал
Узнал фоток 22
Распознаем Керри
все
Записываем видео


In [ ]:
video_path = directory + "out_video_carry4.avi"
mp4 = open(video_path,'rb').read()
decoded_vid = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f'<video width=400 controls><source src={decoded_vid} type="video/mp4"></video>')